In [1]:
from types import SimpleNamespace
from fcn import *
from data import *
from utils import *

## Pandas Dataframe format

* One dataframe for saving latent features with columns

PATIENT ID | CLASS | VALIDATION LATENT FEAUTES
* One dataframe for saving input feature vectors and names

PATIENT ID | CLASS | VALIDATION FEATURES

**All features will be generated using leave-one-out cross validation**

In [2]:
#--- Autoselect GPU
from jarvis.utils.general import gpus
gpus.autoselect()

[ 2021-05-13 02:31:28 ] CUDA_VISIBLE_DEVICES automatically set to: 1           


In [3]:
# find out why chi2 is so high for als no als
# TODO: get and reproduce baselines...
# TODO: add specialized class splitting
# DEBUG: why is mutual info lower in new code

In [4]:
def make_dataset(norm=False, feat_select=False):
    return load_data(norm=norm, feat_select=feat_select)
    
def load_dataset(path='./DESeq2_reduced.pkl'):
    dataset = load(path)
    return SimpleNamespace(**dataset)

def normalize(X):
    X = X - np.min(X, axis=0, keepdims=True)
    X = X / np.max(X, axis=0, keepdims=True)
    return X

In [5]:
# TODO: add feature selection visualizer to compare between mutual information and chi^2.

# The mutual information has a lot more unique feature values, however maybe this variation causes too much noise and chi^2 is better especially because the dataset is small

# The features are very sparse lot's of clustering at 0 and 1!

## Single Loss vs. Multi-Loss

### Datasets
1. DESeq2_reduced.pkl is a dataset with 5 losses with features corresponding to the top 1% of mutual information feature selection for the ctrl vs. case task
2. DESeq2_all_reduced.pkl is a dataset with 5 losses with features corresponding to the top 1% of mutual information feature selection for all tasks
3. DESeq2_all_reduced_tiny.pkl is a dataset with 5 losses with features corresponding to the top .02% of mutual information feature selection for all tasks

In [40]:
# --- Show data for (1.)
dataset = load_dataset('pkls/group/DESeq2_reduced.pkl')
dataset.data

,Participant_ID,CtrlVsCase_Classifier,DDX11L1,WASH7P,MIR6859.1,MIR1302.2HG,FAM138A,OR4G11P,OR4F5,AL627309.1,...,ENSG00000278033,U1.4,U1.5,AC213203.2,AC213203.1,ENSG00000278858,Bulbar_Classifier,Median_Classifier,Limb_Classifier,High_Classifier
0,NEUAB000NKC,1,0.000000,188.401759,32.563267,1.162974,0.0,0.000000,0.0,5.814869,...,0.000000,94.200880,2.325948,5.814869,0.0,0.0,1.0,0.0,0.0,1.0
1,NEUAE993EPR,1,0.000000,233.423189,27.412688,1.661375,0.0,0.000000,0.0,4.984125,...,0.000000,34.888875,0.000000,0.830688,0.0,0.0,0.0,0.0,0.0,1.0
2,NEUAF553MJ3,1,0.000000,199.081434,41.519657,3.193820,0.0,0.000000,0.0,15.969099,...,0.000000,57.488756,3.193820,1.064607,0.0,0.0,0.0,0.0,1.0,0.0
3,NEUAG603XLK,1,2.152546,122.695105,34.440731,2.152546,0.0,2.152546,0.0,8.610183,...,0.000000,60.271280,0.000000,4.305091,0.0,0.0,1.0,0.0,0.0,1.0
4,NEUAM655HF7,1,0.000000,91.017494,19.679458,1.229966,0.0,0.000000,0.0,4.919865,...,0.000000,56.578442,1.229966,0.000000,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NEUUV825HYF,0,0.000000,257.219991,55.316127,0.000000,0.0,0.000000,0.0,9.680322,...,0.000000,34.572579,1.382903,1.382903,0.0,0.0,0.0,0.0,0.0,0.0
159,NEUVZ050YX7,0,0.000000,136.129306,34.032327,2.520913,0.0,1.260457,0.0,10.083652,...,1.260457,60.501914,1.260457,2.520913,0.0,0.0,0.0,0.0,0.0,0.0
160,NEUWT164JRQ,0,0.000000,104.411115,26.102779,0.000000,0.0,0.000000,0.0,12.601342,...,0.000000,116.112361,0.000000,0.900096,0.0,0.0,0.0,0.0,0.0,0.0
161,NEUYM205MRL,0,1.032984,43.385342,12.395812,2.065969,0.0,0.000000,0.0,11.362828,...,0.000000,39.253405,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# --- Train single loss
dataset = load_dataset('pkls/group/DESeq2_reduced.pkl')
dataset.start = 0
dataset.losses = dataset.losses[0:2]
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
CtrlVsCase_Classifier
--- Training Results
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 1.0
--- Validation Results
acc     score: 0.9183673469387755
roc_auc score: 0.9333333333333333
prc_auc score: 0.9844804803861014


In [13]:
# --- Train multi-loss
dataset = load_dataset('pkls/group/DESeq2_reduced.pkl')
dataset.start = 0
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

CtrlVsCase_Classifier
acc     score: 0.7719298245614035
roc_auc score: 0.9605734767025089
prc_auc score: 0.9913362026559122
acc     score: 0.8367346938775511
roc_auc score: 0.9388888888888889
prc_auc score: 0.9862931628108756
Bulbar_Classifier
acc     score: 0.7105263157894737
roc_auc score: 0.889196675900277
prc_auc score: 0.8949488614974064
acc     score: 0.5714285714285714
roc_auc score: 0.575
prc_auc score: 0.5644357228929467
Median_Classifier
acc     score: 0.35964912280701755
roc_auc score: 0.19588029537504856
prc_auc score: 0.17064489258162324
acc     score: 0.5918367346938775
roc_auc score: 0.5408163265306123
prc_auc score: 0.3396739541971839
Limb_Classifier
acc     score: 0.956140350877193
roc_auc score: 0.9841269841269841
prc_auc score: 0.9473154860634914
acc     score: 0.7346938775510204
roc_auc score: 0.4083333333333333
prc_auc score: 0.14372382866464883
High_Classifier
acc     score: 0.45614035087719296
roc_auc score: 0.25396825396825395
prc_auc score: 0.17832625882566275


**Comment**

From these results it is obvious that a feed-forward neural network trained with multi-loss struggles to converge. This may be due to the dataset size and also the fact that mutual information feature selection is being done on only a single task instead of all. 

Thus our next step is to try feature selection on all tasks, and concatenate the resulting features together.

In [19]:
# --- Train multi-loss with feature selection on all losses (p1)
dataset = load_dataset('pkls/group/DESeq2_all_reduced.pkl')
dataset.start = 0
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

CtrlVsCase_Classifier
acc     score: 0.7017543859649122
roc_auc score: 1.0
prc_auc score: 1.0
acc     score: 0.6938775510204082
roc_auc score: 0.7402777777777778
prc_auc score: 0.9054048121398618
Bulbar_Classifier
acc     score: 0.7105263157894737
roc_auc score: 0.9444444444444444
prc_auc score: 0.950928052911302
acc     score: 0.5102040816326531
roc_auc score: 0.6200000000000001
prc_auc score: 0.6593996602363652
Median_Classifier
acc     score: 0.40350877192982454
roc_auc score: 0.09483093664982511
prc_auc score: 0.15512670942412699
acc     score: 0.6122448979591837
roc_auc score: 0.47244897959183674
prc_auc score: 0.2828797692402075
Limb_Classifier
acc     score: 0.8157894736842105
roc_auc score: 0.9654377880184332
prc_auc score: 0.8348609692107325
acc     score: 0.5918367346938775
roc_auc score: 0.5472222222222223
prc_auc score: 0.17918397000040004
High_Classifier
acc     score: 0.35964912280701755
roc_auc score: 0.09722222222222221
prc_auc score: 0.15446956519788688
acc     score: 

In [15]:
# --- Train multi-loss with feature selection on all losses (p2)
dataset = load_dataset('pkls/group/DESeq2_all_reduced_tiny.pkl')
dataset.start = 0
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

CtrlVsCase_Classifier
acc     score: 0.7719298245614035
roc_auc score: 0.8125960061443932
prc_auc score: 0.9594234714480145
acc     score: 0.7142857142857143
roc_auc score: 0.6
prc_auc score: 0.8638393190029524
Bulbar_Classifier
acc     score: 0.6929824561403509
roc_auc score: 0.8165589412126808
prc_auc score: 0.7893395029549168
acc     score: 0.5306122448979592
roc_auc score: 0.6033333333333334
prc_auc score: 0.6712757341590616
Median_Classifier
acc     score: 0.35964912280701755
roc_auc score: 0.22425184609405363
prc_auc score: 0.1746419711595185
acc     score: 0.5306122448979592
roc_auc score: 0.5122448979591837
prc_auc score: 0.3034588863170158
Limb_Classifier
acc     score: 0.9210526315789473
roc_auc score: 0.9810547875064004
prc_auc score: 0.9312930463622622
acc     score: 0.7142857142857143
roc_auc score: 0.5222222222222223
prc_auc score: 0.1865005186822392
High_Classifier
acc     score: 0.35964912280701755
roc_auc score: 0.1615079365079365
prc_auc score: 0.15938136308281556
acc

**Comment**

From these results it is shown that training a neural network on muliple tasks for the given dataset is challenging even when considering feature selection on mulitple targets. The difference between p1 and p2 is that p1 contains ~5x more features than p2. As shown from the results between p1 and p2, performance remained roughly similar even when decreasing the amount of features.

Moreover, the result is not even close to the performance attainable when training on a single loss.

We hypothesize given the nature of the current dataset that multi-loss training is not a feasible approach. Furthermore, we believe that each task has a different difficulty level which makes it even harder for the network to learn. For instance it is possible that the neural network overfits for certain tasks while underfits for another task.

Empirically we find that: 
1. Median low vs High requires the least complex features to separate the data
2. Bulbar vs Limb requires complexity between 1. and 3. to separate the data
3. Ctrl vs Case requires the most complex features to separate the data

Due to these difficulties, we opt for a single loss approach as it yields significantly higher performance.

The rest of our approach relies on training a single network only on specific tasks.

## Single Loss Datasets

Examples to load different datasets are shown below

In [7]:
# --- Bulbar vs. Limb mutual information feature selection
dataset = load_dataset('pkls/individual/DESeq2_bl_mi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

# --- Bulbar vs. Limb chi^2 feature selection
dataset = load_dataset('pkls/individual/DESeq2_bl_chi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
SiteOnset_Class
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 1.0
acc     score: 0.8823529411764706
roc_auc score: 0.9466666666666667
prc_auc score: 0.9796068068891869
  ...
    to  
  ['...']
  ...
    to  
  ['...']
SiteOnset_Class
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 1.0
acc     score: 0.9411764705882353
roc_auc score: 1.0
prc_auc score: 1.0


In [12]:
# --- Ctrl vs. Case mutual information feature selection
dataset = load_dataset('pkls/individual/DESeq2_cc_mi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

# --- Ctrl vs. Case chi^2 feature selection
dataset = load_dataset('pkls/individual/DESeq2_cc_chi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
CtrlVsCase_Classifier
--- Training Results
acc     score: 0.9912280701754386
roc_auc score: 1.0
prc_auc score: 1.0
--- Validation Results
acc     score: 0.8775510204081632
roc_auc score: 0.9083333333333333
prc_auc score: 0.9772878326207415
  ...
    to  
  ['...']
  ...
    to  
  ['...']
CtrlVsCase_Classifier
--- Training Results
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 1.0
--- Validation Results
acc     score: 0.8571428571428571
roc_auc score: 0.7611111111111112
prc_auc score: 0.844631017667826


In [9]:
# --- Median low vs. High mutual information feature selection
dataset = load_dataset('pkls/individual/DESeq2_mh_mi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

# --- Median low vs. High chi^2 feature selection
dataset = load_dataset('pkls/individual/DESeq2_mh_chi.pkl')
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=1, verbose=False)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
ALSFRS_Class_Median
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 0.9999999999999998
acc     score: 0.5925925925925926
roc_auc score: 0.6758241758241759
prc_auc score: 0.7485413756901699
  ...
    to  
  ['...']
  ...
    to  
  ['...']
ALSFRS_Class_Median
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 0.9999999999999998
acc     score: 1.0
roc_auc score: 1.0
prc_auc score: 0.9999999999999998


**Commet**

From these results it is shown that the type of feature selection used had significant impacts depending on the dataset task.

In summary:
* Chi^2 gave better performance on the limb vs bulbar and median low vs high tasks
* Mutual information gave better performance on the ctrl vs case task

*However, across mutliple dataset splits, it was found that Chi^2 gave more numerically stable results, so all following experiments will use Chi^2 feature selection.*

Next we will run leave-one-out cross validation on the each dataset task with the best performing feature selection.


**Correction stratified version of leave-one-out is used fix all above comments that mention this!!!**

## Cross-Validation

Cross validation is performed to get the validation results on all possible training set configurations. 

During each fold, the latent representation of the neural network is saved for clustering.

The following datasets and settings are used for each task.

1. Ctrl vs Case:
    1. Mutual information feature selection
2. Bulbar vs Limb:
    1. Chi^2 feature selection 
3. Median Low vs High
    1. Chi^2 feature selection

In [ ]:
# --- 1. Ctrl vs Case
dataset = Dataset('./data/ctrl_vs_case.csv')
dataset.feature_selection(norm=True, feat_select='mi', target=0, percentile=.01)
n_fold = min(len(dataset.labels) - sum(dataset.labels), sum(dataset.labels))
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=n_fold, verbose=False)

# --- Save dataframe
dataset.lf_df.to_csv('./outputs/ctrl_vs_case_lf.csv' ,index=False)
dataset.ft_df.to_csv('./outputs/ctrl_vs_case_ft.csv' ,index=False)

# Get Accuracy results
acc = lambda x: sum(x)/len(x)
a = []
for h in (history):
    a.append(acc(h.history['val_sparse_categorical_accuracy']))
    
acc(a)

Using mi feature selection and data normalization = True


In [ ]:
# --- 2. Bulbar vs Limb
dataset = Dataset('./data/bulbar_vs_limb.csv')
dataset.feature_selection(norm=True, feat_select='chi', target=0, percentile=.01)
n_fold = min(len(dataset.labels) - sum(dataset.labels), sum(dataset.labels))
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=n_fold, verbose=False)

# --- Save dataframe
dataset.lf_df.to_csv('./outputs/bulbar_vs_limb_lf.csv', index=False)
dataset.ft_df.to_csv('./outputs/bulbar_vs_limb_ft.csv' ,index=False)

# Get Accuracy results
acc = lambda x: sum(x)/len(x)
a = []
for h in (history):
    a.append(acc(h.history['val_sparse_categorical_accuracy']))
    
acc(a)

In [21]:
# --- 1. Median Low vs High
dataset = Dataset('./data/median_low_vs_high.csv')
dataset.feature_selection(norm=True, feat_select='chi', target=0, percentile=.01)
n_fold = min(len(dataset.labels) - sum(dataset.labels), sum(dataset.labels))
history, model = learn(dataset, lr=3e-3, batch_size=64, epochs=100, drop=0.5, n_folds=n_fold, verbose=False)

# --- Save dataframe
dataset.lf_df.to_csv('./outputs/median_low_vs_high_lf.csv', index=False)
dataset.ft_df.to_csv('./outputs/median_low_vs_high_ft.csv', index=False)

# Get Accuracy results
acc = lambda x: sum(x)/len(x)
a = []
for h in (history):
    a.append(acc(h.history['val_sparse_categorical_accuracy']))
    
acc(a)

0.9942063492181754

In [ ]:
model.summary()

## Ignore

In [20]:
# TODO: add special task for using only specific patients
# np.random.seed(0)
# dataset = Dataset(['./data/ctrl_vs_case.csv', './data/bulbar_vs_limb.csv', './data/median_low_vs_high.csv'])
# dataset.feature_selection(norm=True, feat_select='mi', target=0, percentile=.01)
# dataset.split_data(train_size=0.7, start=0) # start only equal 1 when bulbar v lim or med v high

# dataset = Dataset('./data/median_low_vs_high.csv')

# # adjust_dataset, only change non_ attributes
# dataset.feature_selection(norm=True, feat_select='chi', target=0, percentile=.01)
# dataset.split_data(train_size=0.7, start=0) # start only equal 1 when bulbar v lim or med v high


# PCA
dataset = load_dataset('pkls/group/DESeq2_reduced.pkl')
p = pca(dataset, n_components=10)
df, l = pca_importance(dataset, p)
# feature_importance(dataset)

dataset = load_dataset('./DESeq2_reduced.pkl')
p = pca(dataset, n_components=10)
pca_importance(dataset, p)
feature_importance(dataset)

x = p.fit_transform(dataset.features)

## Model Saving

In [ ]:
# --- Analyze model
model.summary()

# --- Save a model
# save_model(model)
# del model

# --- Load a model
# model = load_model()